# Kymograph maker
This notebook will find competitive cellular "events" in the simplest definition (i.e. a loser cell apoptosis) and return information about the spatiotemporal distrubition of counterpart competitive events (i.e. winner cell mitosis) in the form of a kymograph.

kymograph:
/ˈkʌɪmə(ʊ)ɡrɑːf/
_noun_ 
a graphical representation of spatial positions (of wild-type cells or mitoses) over time in which a spatial axis represents time, focused around an apoptotic event.

Contents:

- Load modules
- Set experiment data path
- Load tracking data
- Isolate tracks of interest (target track)
- Iterate over tracks, generating and saving kymographs

To-do:

- [ ] Rewrite this to iterate over many target tracks

In [ ]:
import napari
import btrack
import numpy as np
from skimage.io import imread
import os
print("Napari version no.:", napari.__version__)
print("btrack version no.:", btrack.__version__)
from btrack.utils import import_HDF, import_JSON, tracks_to_napari
from tqdm.notebook import tnrange, tqdm
import matplotlib.pyplot as plt

def find_apoptosis_time(target_track, index): ### if index is set to True then the index of the apoptotic time (wrt target_track) is returned
    for i, j in enumerate(target_track.label):
        if j == 'APOPTOSIS' and target_track.label[i+1] == 'APOPTOSIS' and target_track.label[i+2] == 'APOPTOSIS': # and target_track.label[i+3] =='APOPTOSIS' and target_track.label[i+4] =='APOPTOSIS':
            apop_index = i
            break
    apop_time = target_track.t[apop_index]
    if index == True: 
        return apop_index
    else: 
        return apop_time

### Set experiment data path 

In [ ]:
# print("Input experiment number")
# experiment_no = input()
# root_path = os.path.join('/home/nathan/data/kraken/h2b/giulia/', experiment_no)
root_path = '/home/nathan/data/kraken/h2b/giulia/GV0807'  ## this overwrites input option for ease 
gfp_path = os.path.join(root_path, 'Pos3/stacks/gfp.tif')
rfp_path = os.path.join(root_path, 'Pos3/stacks/rfp.tif')
bf_path = os.path.join(root_path, 'Pos3/stacks/bf.tif')
tracks_path = os.path.join(root_path, 'Pos3/Pos3_aligned/HDF/segmented.hdf5')

### Load image data

In [ ]:
gfp = imread(gfp_path)

In [ ]:
rfp = imread(rfp_path)

In [ ]:
#bf = imread(bf_path) #3 large 1000+ frame stacks is usually too much for memory to process on 16gb ram

### Load tracking data

In [ ]:
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_1") as hdf:
    wt_tracks = hdf.tracks
with btrack.dataio.HDF5FileHandler(tracks_path, 'r', obj_type = "obj_type_2") as hdf:
    scr_tracks = hdf.tracks
print("Tracks loaded")

In [ ]:
scr_apops = [scr_track for scr_track in scr_tracks if scr_track.fate.name == 'APOPTOSIS']

### Set kymograph parameters

In [ ]:
radius = 600
delta_t = 400
num_radial_bins = 20
radial_bin = radius / num_radial_bins
num_temporal_bins = 20
temporal_bin = delta_t / num_temporal_bins
print("size of temporal and radial bins:",temporal_bin,",", radial_bin)

In [ ]:
### generate labels for axis
xlocs = range(0, num_temporal_bins, 2) ## step of 2 to avoid crowding
xlabels = []
for m in range(int(-num_temporal_bins/2), int(num_temporal_bins/2), 2):
    xlabels.append((num_temporal_bins/2)*m)
print("xlabels:",xlabels, type(xlabels))
ylocs = range(0, num_radial_bins, 2) ## step of 2 to avoid crowding
ylabels = []
for m in range(0, num_radial_bins, 2):
    ylabels.append((num_temporal_bins/2)*m)
print("ylabels:", ylabels, type(ylabels))

In [ ]:
for scr_track in scr_apops:
    target_track = scr_track 
    apop_index = find_apoptosis_time(target_track, index = True)
    apop_time = find_apoptosis_time(target_track, index = False) 
    ## instead of going over all tracks, could go over all tracks that are definitely in_frame (using func) or in zone 
    ## set empty data variables to store 
    num_wt, num_wt_mito = np.zeros((num_radial_bins, num_temporal_bins)), np.zeros((num_radial_bins, num_temporal_bins))
    ## load one wt_track
    for j, wt_track in enumerate(tqdm(wt_tracks)):
        print("ID:", wt_track.ID)
        ## load first distance/radial bin
        for n in range(num_radial_bins):
            print("radial", range(int(radial_bin * n), int(radial_bin * (n+1))))
            ## load first temporal bin, over negative and positive range
            for l, m in enumerate(range(int(-num_temporal_bins/2), int(num_temporal_bins/2))): ## l introduced to iterate through pos integers for data storage
                print("time",range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1))))
                ## load first timepoint of wt_track
                for i in range(len(wt_track)): 
                    ## calculate if within euclidean distance: if min of radial distance bin sqrd < (wt_track.x (at first time point) - target_track.x (at apop_time)) both squared + corresponding for yboth squared < max_dist squared
                    if ((radial_bin * n)**2) < (wt_track.x[i] - target_track.x[apop_index])**2 + (wt_track.y[i] - target_track.y[apop_index])**2 < ((radial_bin * (n+1))**2):
                        ## if FIRST time point is in temporal bin (centered around apop_time)
                        if wt_track.t[i] in (range(int(apop_time+temporal_bin*m),int(apop_time+temporal_bin*(m+1)))):
                            num_wt[n,l] += 1 #str(wt_track.ID)
                            print("wt_track", wt_track.ID)
                            #print("WT ID:", wt_track.ID, "time:", wt_track.t[i], "radial bin:", (n*radial_bin),"-", ((n+1)*radial_bin), "temporal bin:", (temporal_bin * m),"-", (temporal_bin * (m+1)),"i,m,n:", i,m,n)
                            if wt_track.fate.name == "DIVIDE" and "'METAPHASE', 'METAPHASE'," in str(wt_track.label[int(apop_time+(temporal_bin*m))-wt_track.t[0]:int(apop_time+(temporal_bin*(m+1)))-wt_track.t[0]]): ##explainer: this double condition states that if wt_track ends in mitosis but also has THREE (?) sequential metaphase classifications within the time window of that temporal bin then the condition is met
                                num_wt_mito[n,l] += 1 #str(wt_track.ID)
                                print("wt_track mitosis", wt_track.ID)
                                #print("MITO ID:", wt\_track.ID, "time:", wt_track.t[i], "radial bin:", (n*radial_bin),"-", ((n+1)*radial_bin), "temporal bin:", (temporal_bin * m),"-", (temporal_bin * (m+1)),"i,m,n:", i,m,n)
                            break ## if track is in bin, break loop iterating over all track positions to skip to next bin
                            
    ## gross wt
    title = "Scr_" + str(scr_track.ID) + "_num_wt.png"
    plt.title('No. of wild-type cells')
    plt.xlabel("time bins")
    plt.ylabel("radial bins")
    plt.yticks(ylocs, ylabels)
    plt.xticks(xlocs, xlabels, rotation='vertical')
    plt.imshow(num_wt)
    plt.colorbar()
    plt.savefig(os.path.join(root_path, title), bbox_inches = 'tight')

    ## mitoses
    title = "Scr_" + str(scr_track.ID) + "_num_wt_mitoses.png"
    plt.title('mitosis kymograph')
    plt.xlabel("time bins")
    plt.ylabel("radial bins")
    plt.ylim(0, num_radial_bins-0.5)
    plt.yticks(ylocs, ylabels)
    plt.xticks(xlocs, xlabels, rotation='vertical')
    plt.imshow(num_wt_mito)
    plt.colorbar()
    plt.savefig(os.path.join(root_path, title), bbox_inches = 'tight')

    ## probability
    title = "Scr_" + str(scr_track.ID) + "_prob_wt_mitoses.png"
    plt.title('Probability of mitosis kymograph')
    plt.xlabel("time bins")
    plt.ylabel("radial bins")
    plt.ylim(0, num_radial_bins-0.5)
    plt.yticks(ylocs, ylabels)
    plt.xticks(xlocs, xlabels, rotation='vertical')
    prob_mito = num_wt_mito / num_wt
    plt.imshow(prob_mito)
    plt.colorbar()
    plt.savefig(os.path.join(root_path, title), bbox_inches = 'tight')

In [ ]:
for scr_track in scr_tracks:
    
    ## gross wt
    title = "Scr_" + str(scr_track.ID) + "_num_wt.png"
    plt.title('No. of wild-type cells')
    plt.xlabel("time bins")
    plt.ylabel("radial bins")
    plt.yticks(ylocs, ylabels)
    plt.xticks(xlocs, xlabels, rotation='vertical')
    plt.imshow(num_wt)
    plt.colorbar()
    plt.savefig(os.path.join(root_path, title), bbox_inches = 'tight')
    
    ## mitoses
    title = "Scr_" + str(scr_track.ID) + "_num_wt_mitoses.png"
    plt.title('mitosis kymograph')
    plt.xlabel("time bins")
    plt.ylabel("radial bins")
    plt.ylim(0, num_radial_bins-0.5)
    plt.yticks(ylocs, ylabels)
    plt.xticks(xlocs, xlabels, rotation='vertical')
    plt.imshow(num_wt_mito)
    plt.colorbar()
    plt.savefig(os.path.join(root_path, title), bbox_inches = 'tight')

    ## probability
    title = "Scr_" + str(scr_track.ID) + "_prob_wt_mitoses.png"
    plt.title('Probability of mitosis kymograph')
    plt.xlabel("time bins")
    plt.ylabel("radial bins")
    plt.ylim(0, num_radial_bins-0.5)
    plt.yticks(ylocs, ylabels)
    plt.xticks(xlocs, xlabels, rotation='vertical')
    prob_mito = num_wt_mito / num_wt
    plt.imshow(prob_mito)
    plt.colorbar()
    plt.savefig(os.path.join(root_path, title), bbox_inches = 'tight')


# need to write smthn to save out kymographs with unique titles